# archive to folder

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/mlrun"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/mlrun'


In [3]:
import os
import zipfile
import urllib.request
import tarfile
import json

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import Union

def open_archive(
    context: MLClientCtx, 
    archive_url: DataItem,
    subdir: str = "content",
    key: str = "content"
):
    """Open a file/object archive into a target directory
    
    Currently supports zip and tar.gz
    
    :param context:      function execution context
    :param archive_url:  url of archive file
    :param subdir:       path within artifact store where extracted files
                         are stored
    :param key:          key of archive contents in artifact store
    """
    os.makedirs(subdir, exist_ok=True)
    
    archive_url = archive_url.local()
    if archive_url.endswith("gz"):
        with tarfile.open(archive_url, mode="r|gz") as ref:
            ref.extractall(subdir)
    elif archive_url.endswith("zip"):
        with zipfile.ZipFile(archive_url, "r") as ref:
            ref.extractall(subdir)
    else:
        raise ValueError(f'unsupported archive type in {archive_url}')
    
    context.log_artifact(key, local_path=subdir)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function, run_local
# create job function object from notebook code
fn = code_to_function("open_archive")

# add metadata (for templates and reuse)
fn.spec.default_handler = "open_archive"
fn.spec.description = "Open a file/object archive into a target directory"
fn.metadata.categories = ["data-movement", "utils"]
fn.metadata.labels = {"author": "yaronh"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-05-03 15:18:29,902 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:18:29,925 function spec saved to path: function.yaml


## tests

In [7]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

### tar

In [8]:
run = run_local(
    handler=open_archive, 
    inputs={'archive_url': "https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz"})

[mlrun] 2020-05-03 15:18:29,953 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:18:29,954 starting run mlrun-b9deef-open_archive uid=5dfdc84fff9949198577a7a069a20402  -> http://mlrun-api:8080
[mlrun] 2020-05-03 15:18:29,972 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:18:29,985 downloading https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz to local tmp
[mlrun] 2020-05-03 15:18:43,541 log artifact content at /User/artifacts/content/, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...69a20402,0,May 03 15:18:29,completed,mlrun-b9deef-open_archive,v3io_user=adminkind=handlerowner=adminhost=jupyter-6c5fccf844-gxlrw,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run 5dfdc84fff9949198577a7a069a20402 --project default , !mlrun logs 5dfdc84fff9949198577a7a069a20402 --project default
[mlrun] 2020-05-03 15:18:43,621 run executed, status=completed


### zip

In [9]:
run_local(
    handler=open_archive, 
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'})

[mlrun] 2020-05-03 15:18:43,636 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:18:43,637 starting run mlrun-96d000-open_archive uid=fbbaedbef3fd4638ae80e83963da8cda  -> http://mlrun-api:8080
[mlrun] 2020-05-03 15:18:43,655 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 15:18:43,666 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-05-03 15:18:52,818 log artifact content at /User/artifacts/content/, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...63da8cda,0,May 03 15:18:43,completed,mlrun-96d000-open_archive,v3io_user=adminkind=handlerowner=adminhost=jupyter-6c5fccf844-gxlrw,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run fbbaedbef3fd4638ae80e83963da8cda --project default , !mlrun logs fbbaedbef3fd4638ae80e83963da8cda --project default
[mlrun] 2020-05-03 15:18:52,852 run executed, status=completed
